# Part 1: Clean the NYC real estate data (AirBnB is already cleaned)

In [2]:
#import io
import pandas as pd
import numpy as np
#from google.colab import files
#uploaded = files.upload()

realEstateData = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Raw Data\cost_adjusted_nyc-rolling-sales.csv')

## Replace all ' -  's to null and drop the rows
# clean1 is realEstateData with values of '-' replaced into NaN
clean1 = realEstateData.copy()
clean1 = clean1.replace(' -  ', np.nan)

# clean2 is clean1 with all rows contains NaN dropped.
clean2 = clean1.copy()
clean2 = clean2.dropna()

# Filter rows with sales price of 100k and more
# clean3 is clean2 with sales price of 100k and more
clean3 = clean2.copy()
clean3['COST ADJUSTED SALE PRICE'] = clean3['COST ADJUSTED SALE PRICE'].apply(pd.to_numeric)
clean3 = clean3.loc[clean3['COST ADJUSTED SALE PRICE'] >= 100000]

# Filter values in YEAR BUILT anytime after year of 1618
# clean4 is clean3 with YEAR BUILT anytime after year of 1618
clean4 = clean3.copy()
clean4['YEAR BUILT'] = clean4['YEAR BUILT'].apply(pd.to_numeric)
clean4 = clean3.loc[clean4['YEAR BUILT'] > 1618]

#For Building Class, I have went with the route to filter out all business properties and rental/coops.
#A list of values in the Building Class Categories in the underlying data can be found here: 
#(https://1drv.ms/x/s!Aqw31IAta2jYg89CVfP4UvWRsiuQgg?e=1wBQsh)
# Filter all unwanted rows containing certain building classes.
# Crop entries in BUILDING CLASS CATEGORY (BCC) and only save first 2 digits as identifying codes.
# clean5 is clean4 with clipped BCC values.
clean5 = clean4.copy()
clean5['BUILDING CLASS CATEGORY'] = clean5['BUILDING CLASS CATEGORY'].astype(str)
clean5['BUILDING TYPE CODE'] = clean5['BUILDING CLASS CATEGORY'].str[0:2]
clean5['BUILDING TYPE'] = clean5['BUILDING CLASS CATEGORY'].str[2:len(clean5['BUILDING CLASS CATEGORY'])]

# Select and save data only with ID code: 01, 02, 03, 04, 12, 13
# clean6 is clean5 with only clipped BCC values of 01, 02, 03, 04, 12, 13
clean6 = clean5.copy()
clean6 = clean6.loc[(clean6['BUILDING TYPE CODE'] == '01') |
                    (clean6['BUILDING TYPE CODE'] == '02') |
                    (clean6['BUILDING TYPE CODE'] == '03') |
                    (clean6['BUILDING TYPE CODE'] == '04') |
                    (clean6['BUILDING TYPE CODE'] == '12') |
                    (clean6['BUILDING TYPE CODE'] == '13')]

#Recoding borough data:
# clean7 is clean6 with recoded borough data
clean7 = clean6.copy()
clean7.reset_index(drop=True,inplace=True)

# change data type of borough into string
clean7['BOROUGH'] = clean7['BOROUGH'].apply(str)

#create BUROUGH NAME empty list for use in loop
borough_name = []

#Populate borough_name
for i in range(len(clean7['BOROUGH'])):
    if clean7['BOROUGH'][i] == '1':
        borough_name.append('Manhattan')
    elif clean7['BOROUGH'][i] == '2':
        borough_name.append('Bronx')
    elif clean7['BOROUGH'][i] == '3':
        borough_name.append('Brooklyn')
    elif clean7['BOROUGH'][i] == '4':
        borough_name.append('Queens')
    elif clean7['BOROUGH'][i] == '5':
        borough_name.append('Staten Island')
    
#make borough_name a new column in the dataframe
clean7['BOROUGH NAME'] = borough_name

## At this stage, clean7 is the current cleaned database.
#Export as csv
clean7.to_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\NYC_Real_Estate_Cleaned.csv',index=False)

# Process several files' data to determine the number of subway stops near each property

## Working with longitude and latitude on the NYC Subway data

In [1]:
#Import needed modules and the data
import pandas as pd
import numpy as np
import math as math
stations = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Raw Data\NYC_Subway_Data.csv')

#extract longitude and latitude and rejoin the resulting dataframe with the original
stations_2 = stations['the_geom'].str.replace('(','').str.replace(')','').str.split(expand=True).rename(columns={0:'Junk', 1:'Longitude',2:'Latitude'})
stations_3 = pd.concat([stations, stations_2],axis= 1)

# Convert coordinates to floats and drop the crap columns
stations_3['Longitude'] = stations_3['Longitude'].astype('float64')
stations_3['Latitude'] = stations_3['Latitude'].astype('float64')
stations_4 = stations_3.drop(['URL'], axis = 1).drop(['the_geom'], axis = 1).drop(['Junk'], axis = 1)

#Further convert longitude and Latitude to degrees, minutes, and miles

#Per information found at http://astro.unl.edu/naap/motion1/tc_units.html#:~:text=Degrees%2C%20Minutes%2C%20Seconds,into%2060%20seconds%20(%E2%80%9D).
degrees_to_minutes_convert = 60

#Per information found at https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps?qt-news_science_products=0#qt-news_science_products
miles_for_latitude_degree = 69
miles_for_latitude_minute = 1.15
miles_for_longitude_degree = 54.6
miles_for_longitude_minute = .91

stations_4['Longitude Degrees'] = np.ceil(stations_4["Longitude"])
stations_4['Longitude Minutes'] = (stations_4["Longitude"] - stations_4['Longitude Degrees']) * degrees_to_minutes_convert
stations_4['Longitude Miles'] = (stations_4['Longitude Degrees'] * miles_for_longitude_degree) + (stations_4['Longitude Minutes'] * miles_for_longitude_minute)

stations_4['Latitude Degrees'] = np.floor(stations_4["Latitude"])
stations_4['Latitude Minutes'] = (stations_4["Latitude"] - stations_4['Latitude Degrees']) * degrees_to_minutes_convert
stations_4['Latitude Miles'] = (stations_4['Latitude Degrees'] * miles_for_latitude_degree) + (stations_4['Latitude Minutes'] * miles_for_latitude_minute)

stations_4.head()

#Export file
stations_4.to_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\NYC_Subway_Coordinates.csv', index = False)

## Working with longitude and latitude, and finding distance to subway stops, on the AirBnB data

In [2]:
#Import needed modules and the data
import pandas as pd
import numpy as np
import math as math
from datetime import datetime

#Please note that in the underlying raw data file, several rows were shifted to the right, causing odd values to appear
#in the date column. Python however, thinks that no data is shifted and as such will not recognize anything when I
# ask it to drop zeros in the date column. Therefore, I have opted to manually cut down the file to 2016 and 2017 in Excel prior to import
airbnb = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Raw Data\Cost_Adjusted_AirBnB_NYC_2016_and_2017_manually_isolated.csv',encoding = 'unicode_escape')

#Convert longitude and Latitude to degrees, minutes, and miles
#Per information found at http://astro.unl.edu/naap/motion1/tc_units.html#:~:text=Degrees%2C%20Minutes%2C%20Seconds,into%2060%20seconds%20(%E2%80%9D).
degrees_to_minutes_convert = 60

#Per information found at https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps?qt-news_science_products=0#qt-news_science_products
miles_for_latitude_degree = 69
miles_for_latitude_minute = 1.15
miles_for_longitude_degree = 54.6
miles_for_longitude_minute = .91

airbnb['Longitude Degrees'] = np.ceil(airbnb["longitude"])
airbnb['Longitude Minutes'] = (airbnb["longitude"] - airbnb['Longitude Degrees']) * degrees_to_minutes_convert
airbnb['Longitude Miles'] = (airbnb['Longitude Degrees'] * miles_for_longitude_degree) + (airbnb['Longitude Minutes'] * miles_for_longitude_minute)

airbnb['Latitude Degrees'] = np.floor(airbnb["latitude"])
airbnb['Latitude Minutes'] = (airbnb["latitude"] - airbnb['Latitude Degrees']) * degrees_to_minutes_convert
airbnb['Latitude Miles'] = (airbnb['Latitude Degrees'] * miles_for_latitude_degree) + (airbnb['Latitude Minutes'] * miles_for_latitude_minute)

#Clean data a little bit more by dropping now needless longitude and latitude measures and renaming some columns
airbnb.drop(['Longitude Degrees','Longitude Minutes','Latitude Degrees','Latitude Minutes'],axis=1,inplace=True)
airbnb.rename(columns={"Longitude Miles":"Airbnb Longitude Miles","Latitude Miles":"Airbnb Latitude Miles"},inplace=True)

airbnb.reset_index(drop=True, inplace = True)

#Now, we attempt Python Magic! For every Airbnb property entry, take its longitude and latitude in miles, which is just a coordinate point on the globe,
#and find the distance, using the distance formula, to every subway stop. Then, count the number of stops within a specified
#"arbitrary mile radius" and deposit that count next to the Airbnb property

arbitrary_mile_radius = .622 #This is equal to one kilometer
success_count_list = []

for i in range(len(airbnb['Airbnb Longitude Miles'])):
    success_count = 0
    for j in range(len(stations_4['Longitude Miles'])):  
        #This next line is the distance formula: square root of ((x2 - x1)^2 + (y2 - y1)^2)
        if math.sqrt((airbnb.at[i,'Airbnb Longitude Miles'] - stations_4.at[j,'Longitude Miles'])**2 + (airbnb.at[i,'Airbnb Latitude Miles'] - stations_4.at[j,'Latitude Miles'])**2) < arbitrary_mile_radius:
            success_count += 1
    success_count_list.append(success_count)

airbnb['Number of Subway Stops Near Each AirBnB Property'] = success_count_list

airbnb.head()

airbnb.to_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\AirBnB_With_Subway_Stops.csv', index = False)

## Working with longitude and latitude, and finding distance to subway stops, on the NYC Real Estate data

In [4]:
#Import needed modules and the data
import pandas as pd
import numpy as np
import math as math
from datetime import datetime

zip_code_coordinates = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Raw Data\zip_code_coordinates.csv')
NYC_real_estate = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\NYC_Real_Estate_Cleaned.csv')

#renaming Zip Code column as "ZIP" in NYC data set in order to synchronize with the zip codes data set
NYC_real_estate.rename(columns={"ZIP CODE":"ZIP"},inplace=True)

#merging NYC data with zip code data so that each zip code in the NYC data gets a longitude and latitude
NYC_real_estate_with_zip_codes = NYC_real_estate.merge(zip_code_coordinates,on = "ZIP")

#Convert longitude and Latitude to degrees, minutes, and miles (reproduced below from above only for reference)
#Per information found at http://astro.unl.edu/naap/motion1/tc_units.html#:~:text=Degrees%2C%20Minutes%2C%20Seconds,into%2060%20seconds%20(%E2%80%9D).
#degrees_to_minutes_convert = 60

#Per information found at https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps?qt-news_science_products=0#qt-news_science_products
#(reproduced below from above only for reference)
#miles_for_latitude_degree = 69
#miles_for_latitude_minute = 1.15
#miles_for_longitude_degree = 54.6
#miles_for_longitude_minute = .91

NYC_real_estate_with_zip_codes['Longitude Degrees'] = np.ceil(NYC_real_estate_with_zip_codes["LNG"])
NYC_real_estate_with_zip_codes['Longitude Minutes'] = (NYC_real_estate_with_zip_codes["LNG"] - NYC_real_estate_with_zip_codes['Longitude Degrees']) * degrees_to_minutes_convert
NYC_real_estate_with_zip_codes['Longitude Miles'] = (NYC_real_estate_with_zip_codes['Longitude Degrees'] * miles_for_longitude_degree) + (NYC_real_estate_with_zip_codes['Longitude Minutes'] * miles_for_longitude_minute)

NYC_real_estate_with_zip_codes['Latitude Degrees'] = np.floor(NYC_real_estate_with_zip_codes["LAT"])
NYC_real_estate_with_zip_codes['Latitude Minutes'] = (NYC_real_estate_with_zip_codes["LAT"] - NYC_real_estate_with_zip_codes['Latitude Degrees']) * degrees_to_minutes_convert
NYC_real_estate_with_zip_codes['Latitude Miles'] = (NYC_real_estate_with_zip_codes['Latitude Degrees'] * miles_for_latitude_degree) + (NYC_real_estate_with_zip_codes['Latitude Minutes'] * miles_for_latitude_minute)

#Clean data a little bit more by dropping now needless longitude and latitude measures and renaming some columns
NYC_real_estate_with_zip_codes.drop(['Unnamed: 0','Longitude Degrees','Longitude Minutes','Latitude Degrees','Latitude Minutes','EASE-MENT'],axis=1,inplace=True)
NYC_real_estate_with_zip_codes.rename(columns={"Longitude Miles":"NYC Real Estate Longitude Miles","Latitude Miles":"NYC Real Estate Latitude Miles"},inplace=True)

NYC_real_estate_with_zip_codes = NYC_real_estate_with_zip_codes.reset_index(drop=True)

#Now, we attempt Python Magic! Again! For every Airbnb property entry, take its longitude and latitude in miles, which is just a coordinate point on the globe,
#and find the distance, using the distance formula, to every subway stop. Then, count the number of stops within a specified
#"arbitrary mile radius" and deposit that count next to the NYC property

arbitrary_mile_radius_2 = .622 #This is equal to one kilometer
success_count_list_2 = []

for i in range(len(NYC_real_estate_with_zip_codes['NYC Real Estate Longitude Miles'])):
    success_count_2 = 0
    for j in range(len(stations_4['Longitude Miles'])):  
        #This next line is the distance formula: square root of ((x2 - x1)^2 + (y2 - y1)^2)
        if math.sqrt((NYC_real_estate_with_zip_codes.at[i,'NYC Real Estate Longitude Miles'] - stations_4.at[j,'Longitude Miles'])**2 + (NYC_real_estate_with_zip_codes.at[i,'NYC Real Estate Latitude Miles'] - stations_4.at[j,'Latitude Miles'])**2) < arbitrary_mile_radius_2:
            success_count_2 += 1
    success_count_list_2.append(success_count_2)

NYC_real_estate_with_zip_codes['Number of Subway Stops Near Each Property'] = success_count_list_2

NYC_real_estate_with_zip_codes.head()

NYC_real_estate_with_zip_codes.to_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\NYC_Real_Estate_With_Subway_Stops.csv', index = False)

# Data setup, numerical standardization, and dummy coding for AirBnB

In [1]:
#Import Needed Packages
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import statistics as stat
from sklearn import metrics

#read in the data
df_airbnb = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\AirBnB_With_Subway_Stops.csv')

#drop the single dependent variable observation where the price = 0
df_airbnb.drop(df_airbnb[df_airbnb['price'] == 0].index, inplace = True)

#drop unneeded columns
df_airbnb.drop(['Cost percentage per listing','price','id','name','host_id','host_name','neighbourhood','latitude','longitude','last_review','calculated_host_listings_count', 
        'availability_365','Airbnb Longitude Miles','Airbnb Latitude Miles','number_of_reviews'],axis=1, inplace=True)

#Group variables into a list based on type
cvar_list_airbnb = ['neighbourhood_group','room_type']
nvar_list_airbnb = ['Gross Income from property per listing','minimum_nights','reviews_per_month','Number of Subway Stops Near Each AirBnB Property']

#standardizing numerical values for use in model
standardized_data_airbnb = df_airbnb.copy()

original_column_values_airbnb = df_airbnb[nvar_list_airbnb]
sample_mean_airbnb = df_airbnb[nvar_list_airbnb].mean()
sample_stddev_airbnb = df_airbnb[nvar_list_airbnb].std()

standardized_data_airbnb[nvar_list_airbnb] = standardized_data_airbnb[nvar_list_airbnb].astype('float64')

standardized_data_airbnb[nvar_list_airbnb] = ((original_column_values_airbnb - sample_mean_airbnb)/sample_stddev_airbnb)

#Creating Dummies for Categorical Variables
standardized_data_airbnb[cvar_list_airbnb] = standardized_data_airbnb[cvar_list_airbnb].astype('category')
standardized_data_airbnb = pd.get_dummies(standardized_data_airbnb, prefix_sep = '_')

df2_airbnb = standardized_data_airbnb.copy()

#Data Partition:
#Splitting the data into our partitions will return two dataframes, so we must prep like so:
testpart_size_airbnb = .2
df_partition_airbnb = df2_airbnb.copy()

df_nontestdata_airbnb, df_testdata_airbnb = train_test_split(df_partition_airbnb, test_size = testpart_size_airbnb, random_state = 1)
df2_airbnb.head()

,minimum_nights,reviews_per_month,Gross Income from property per listing,Number of Subway Stops Near Each AirBnB Property,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,2.402097,0.876339,-0.277178,0.083653,0,1,0,0,0,0,1,0
1,-0.262722,3.167154,-0.213674,-0.149094,0,0,1,0,0,0,1,0
2,0.233058,1.225785,0.023643,0.549148,0,0,1,0,0,1,0,0
3,-0.200749,-0.288483,-0.093345,-1.312829,0,1,0,0,0,1,0,0
4,0.047141,0.216273,-0.293879,-0.149094,0,1,0,0,0,0,1,0


# Data setup, numerical standardization, and dummy coding for NYC Real Estate

In [1]:
#Import Needed Packages
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import statistics as stat
from sklearn import metrics

#read in the data
df_nyc = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Python Exports\NYC_Real_Estate_With_Subway_Stops.csv')

#drop unneeded columns
df_nyc.drop(['BOROUGH','NEIGHBORHOOD','BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT','BLOCK','LOT',
             'BUILDING CLASS AT PRESENT','ADDRESS','APARTMENT NUMBER','ZIP', 
            'TOTAL UNITS','BUILDING CLASS AT TIME OF SALE','SALE DATE','LAT','LNG','NYC Real Estate Longitude Miles',
             'NYC Real Estate Latitude Miles','BUILDING TYPE CODE','SALE PRICE','COST %'],axis=1, inplace=True)

#Group variables into a list based on type
cvar_list_nyc = ['TAX CLASS AT TIME OF SALE','BUILDING TYPE','BOROUGH NAME']
nvar_list_nyc = ['COST ADJUSTED SALE PRICE','RESIDENTIAL UNITS','COMMERCIAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET',
                 'YEAR BUILT','Number of Subway Stops Near Each Property']

#standardizing numerical values for use in model
standardized_data_nyc = df_nyc.copy()

original_column_values_nyc = df_nyc[nvar_list_nyc]
sample_mean_nyc = df_nyc[nvar_list_nyc].mean()
sample_stddev_nyc = df_nyc[nvar_list_nyc].std()

standardized_data_nyc[nvar_list_nyc] = ((original_column_values_nyc - sample_mean_nyc)/sample_stddev_nyc)

standardized_data_nyc[nvar_list_nyc] = standardized_data_nyc[nvar_list_nyc].astype('float64')

#Creating Dummies for Categorical Variables
standardized_data_nyc[cvar_list_nyc] = standardized_data_nyc[cvar_list_nyc].astype('category')
standardized_data_nyc = pd.get_dummies(standardized_data_nyc, prefix_sep = '_')

df2_nyc = standardized_data_nyc.copy()

#Data Partition:
#Splitting the data into our partitions will return two dataframes, so we must prep like so:
testpart_size_nyc = .2
df_partition_nyc = df2_nyc.copy()

df_nontestdata_nyc, df_testdata_nyc = train_test_split(df_partition_nyc, test_size = testpart_size_nyc, random_state = 1)
df2_nyc.head()

,RESIDENTIAL UNITS,COMMERCIAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,COST ADJUSTED SALE PRICE,Number of Subway Stops Near Each Property,TAX CLASS AT TIME OF SALE_1,TAX CLASS AT TIME OF SALE_2,BUILDING TYPE_ CONDOS - ELEVATOR APARTMENTS,BUILDING TYPE_ CONDOS - WALKUP APARTMENTS,BUILDING TYPE_ ONE FAMILY DWELLINGS,BUILDING TYPE_ TAX CLASS 1 CONDOS,BUILDING TYPE_ THREE FAMILY DWELLINGS,BUILDING TYPE_ TWO FAMILY DWELLINGS,BOROUGH NAME_Bronx,BOROUGH NAME_Brooklyn,BOROUGH NAME_Manhattan,BOROUGH NAME_Queens,BOROUGH NAME_Staten Island
0,-0.821079,-0.157310,-0.392310,2.205567,-1.305068,6.614819,3.657612,1,0,0,0,1,0,0,0,0,0,1,0,0
1,0.715302,6.219301,-0.307208,3.386482,-0.750944,6.939435,3.657612,1,0,0,0,0,0,0,1,0,0,1,0,0
2,2.251683,-0.157310,-0.152080,2.068863,-1.305068,7.488786,3.657612,1,0,0,0,0,0,1,0,0,0,1,0,0
3,-0.821079,-0.157310,-0.313043,5.512760,-1.334232,19.824210,3.657612,1,0,0,0,1,0,0,0,0,0,1,0,0
4,-0.821079,6.219301,-0.338817,2.871212,-1.363397,10.984655,3.657612,1,0,0,0,1,0,0,0,0,0,1,0,0


# KNN for NYC Real Estate Data

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# Separate the predictor values and the DV values into X and y respectively
# Placeholder variable: DV
DV = 'COST ADJUSTED SALE PRICE'
y_nyc = df_nontestdata_nyc[DV]
x_nyc = df_nontestdata_nyc.drop(columns=[DV])

y_nyc2 = df_testdata_nyc[DV]
x_nyc2 = df_testdata_nyc.drop(columns = [DV])

# Run Nearest Neighbors with k-fold cross validation with k=5
# Placeholder variable: kfolds
kfolds = 5

# Here we specify within which range of Ks we will search through
max_k = 200
param_grid = {'n_neighbors': list(range(1, max_k+1))}

############ Nearest Neighbors Over the Validation Partition: Run model and find optimal k ######################
gridsearch = GridSearchCV(KNeighborsRegressor(metric = 'euclidean'), param_grid, cv=kfolds, n_jobs=-1)
gridsearch.fit(x_nyc,y_nyc)
clf_bestKNN = gridsearch.best_estimator_
optimal_k = clf_bestKNN.n_neighbors

#ASE over the validation partition
y_nyc_nontest_predicted_CV = clf_bestKNN.predict(x_nyc)
n_obs_nontest_CV = df_nontestdata_nyc.shape[0]

ASE_nontest_CV = sum((y_nyc - y_nyc_nontest_predicted_CV)**2)/n_obs_nontest_CV
print('\n','The optimal k over the validation partition is',optimal_k)
print ("The ASE over the",kfolds,"fold validation partition is",ASE_nontest_CV)

############ Nearest Neighbors Over the Test Partition: Run model and find optimal k ######################
gridsearch2 = GridSearchCV(KNeighborsRegressor(metric = 'euclidean'), param_grid, cv=kfolds, n_jobs=-1)
gridsearch2.fit(x_nyc2,y_nyc2)
clf_bestKNN2 = gridsearch2.best_estimator_
optimal_k2 = clf_bestKNN2.n_neighbors

#ASE over the test partition
y_nyc_test_predicted_CV = clf_bestKNN2.predict(x_nyc2)
n_obs_test_CV = df_testdata_nyc.shape[0]

ASE_test_CV = sum((y_nyc2 - y_nyc_test_predicted_CV)**2)/n_obs_test_CV
print('\n','The optimal k over the test partition is',optimal_k2)
print ("The ASE over the",kfolds,"fold test partition is",ASE_test_CV)




 The optimal k over the validation partition is 9
The ASE over the 5 fold validation partition is 0.40385065314919877

 The optimal k over the test partition is 4
The ASE over the 5 fold test partition is 0.2627613241939576


# KNN for AirBnB Data

In [5]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# Separate the predictor values and the DV values into X and y respectively
# Placeholder variable: DV
DV = 'Gross Income from property per listing'
y_airbnb = df_nontestdata_airbnb[DV]
x_airbnb = df_nontestdata_airbnb.drop(columns=[DV])

y_airbnb2 = df_testdata_airbnb[DV]
x_airbnb2 = df_testdata_airbnb.drop(columns = [DV])

# Run Nearest Neighbors with k-fold cross validation with k=5
# Placeholder variable: kfolds
kfolds = 5

# Here we specify within which range of Ks we will search through
max_k = 200
param_grid = {'n_neighbors': list(range(1, max_k+1))}

############ Nearest Neighbors Over the Validation Partition: Run model and find optimal k ######################
gridsearch = GridSearchCV(KNeighborsRegressor(metric = 'euclidean'), param_grid, cv=kfolds, n_jobs=-1)
gridsearch.fit(x_airbnb,y_airbnb)
clf_bestKNN = gridsearch.best_estimator_
optimal_k = clf_bestKNN.n_neighbors

#ASE over the validation partition
y_airbnb_nontest_predicted_CV = clf_bestKNN.predict(x_airbnb)
n_obs_nontest_CV = df_nontestdata_airbnb.shape[0]

ASE_nontest_CV = sum((y_airbnb - y_airbnb_nontest_predicted_CV)**2)/n_obs_nontest_CV
print('\n','The optimal k over the validation partition is',optimal_k)
print ("The ASE over the",kfolds,"fold validation partition is",ASE_nontest_CV)

############ Nearest Neighbors Over the Test Partition: Run model and find optimal k ######################
gridsearch2 = GridSearchCV(KNeighborsRegressor(metric = 'euclidean'), param_grid, cv=kfolds, n_jobs=-1)
gridsearch2.fit(x_airbnb2,y_airbnb2)
clf_bestKNN2 = gridsearch2.best_estimator_
optimal_k2 = clf_bestKNN2.n_neighbors

#ASE over the test partition
y_airbnb_test_predicted_CV = clf_bestKNN2.predict(x_airbnb2)
n_obs_test_CV = df_testdata_airbnb.shape[0]

ASE_test_CV = sum((y_airbnb2 - y_airbnb_test_predicted_CV)**2)/n_obs_test_CV
print('\n','The optimal k over the test partition is',optimal_k2)
print ("The ASE over the",kfolds,"fold test partition is",ASE_test_CV)


 The optimal k over the validation partition is 168
The ASE over the 5 fold validation partition is 0.7574999902115507

 The optimal k over the test partition is 200
The ASE over the 5 fold test partition is 1.7693450762494152


# Predicting Price of New (Meaning A Tiny Subset of historical) NYC Real Estate Data

In [3]:
# Load the new made-up NYC Real Estate data
df_newdata_nyc = pd.read_csv(r'C:\Users\Home\Documents\Data Mining\Group Project\Raw Data\NYC_Real_Estate_With_Subway_Stops_Fake_New_Data.csv')

# Generate the numerical predictor list
npredictor_list_nyc = nvar_list_nyc.copy()
npredictor_list_nyc.remove(DV)

# Set the datatypes of the variables in the made-up data and drop the dependent variable (since we would not know it in real future data)
df_newdata_nyc2 = df_newdata_nyc.copy()
df_newdata_nyc2[cvar_list_nyc] = df_newdata_nyc[cvar_list_nyc].astype('category')
df_newdata_nyc2[npredictor_list_nyc] = df_newdata_nyc[npredictor_list_nyc].astype('float64')
removed_historical_prices = df_newdata_nyc2.pop(DV)

# Use the historical sample mean and historical sample standard deviation to standardize the made-up data
df_newdata_nyc3 = df_newdata_nyc2.copy()
df_newdata_nyc3[npredictor_list_nyc] = (df_newdata_nyc2[npredictor_list_nyc] - sample_mean_nyc[npredictor_list_nyc])/sample_stddev_nyc[npredictor_list_nyc]

# Code the categorical variables in the made-up data 
df_newdata_nyc4 = pd.get_dummies(df_newdata_nyc3, prefix_sep='_')

# Score the new data using the model carried by the model object clf_optimal 
predicted_standardized_price_nyc = clf_bestKNN2.predict(df_newdata_nyc4)

# Convert the standardized predicted prices back to the original predicted prices using the historical sample mean and historical sample standard deviation
predicted_price_nyc = predicted_standardized_price_nyc * sample_stddev_nyc[DV] + sample_mean_nyc[DV]

# Print the predicted prices (in the original dollar amount) for the new data observations 
print(predicted_price_nyc)

[3448472.48   8841657.31   9480472.705  5653233.575  8841657.31
  532675.9325  322957.925   440499.9625  262083.9075  349822.0925
  274521.0225  956527.12    956527.12    956527.12   1085420.845
  563672.615   754141.36    508414.0625  620951.1775  816326.9275
  594561.8825  465374.19    677483.51    503906.6275  340098.5325
  496693.1025]


In [5]:
#Use a naive average to predict the future prices
standardized_historical_prices = standardized_data_nyc[DV]
prediction_by_average = df_nyc[DV].mean()
standardized_prediction_by_average = standardized_historical_prices.mean()

#Calculate ASE for the naive average model
ASE_naive = sum((standardized_data_nyc[DV] - standardized_prediction_by_average)**2)/len(standardized_data_nyc[DV])
print('Using only a naive "averages" model, the standardized error is ',ASE_naive)

#Calculate ASE for the KNN model
standardized_removed_historical_prices = (removed_historical_prices - df_nyc[DV].mean()) / df_nyc[DV].std()

ASE_KNN = sum((standardized_removed_historical_prices - predicted_standardized_price_nyc )**2)/len(standardized_removed_historical_prices)
print('However, using KNN improves the standardized error to ',ASE_KNN)

#Compare the ratio of the improvement
ASE_ratios = ASE_naive/ASE_KNN
print('\nPut differently, and readibly, the naive model is ',ASE_ratios,'times worse than the KNN model.')

#Calculate monetary improvement of KNN model
clean_list = []
KNN_versus_Naive_absolute_dollar_difference = predicted_price_nyc - prediction_by_average
print('\nIn terms of monetary value, the improvements the KNN model offers over the naive averages model, whether the correction is positive or negative, are:')
for i in KNN_versus_Naive_absolute_dollar_difference:
    clean_list.append(round(i,2))
for i in sorted(clean_list,reverse=True):
    print('$',i)

Using only a naive "averages" model, the standardized error is  0.9999620464550201
However, using KNN improves the standardized error to  0.9874570406027728

Put differently, and readibly, the naive model is  1.0126638479833145 times worse than the KNN model.

In terms of monetary value, the improvements the KNN model offers over the naive averages model, whether the correction is positive or negative, are:
$ 8754719.55
$ 8115904.16
$ 8115904.16
$ 4927480.42
$ 2722719.33
$ 359667.69
$ 230773.97
$ 230773.97
$ 230773.97
$ 90573.78
$ 28388.21
$ -48269.64
$ -104801.97
$ -131191.27
$ -162080.54
$ -193077.22
$ -217339.09
$ -221846.52
$ -229060.05
$ -260378.96
$ -285253.19
$ -375931.06
$ -385654.62
$ -402795.23
$ -451232.13
$ -463669.24
